#**Live Code 3**

##**Identity**

Ihsan Aulia Rahman

Full Time Data Science

HCK-Batch-001

#**A. Description**

##**A. 1. Problem Statement**

A travel insurance company has commissioned your help to assess it's claim data. It is seeking information regarding approved insurance claims. 

The company wants to know :
1. Which agencies filed insurance claims?
2. What types of agency that had insurance claims approved?
3. Which country destination received the highest number of claim approval?
4. How old are the approved claim holders?
5. How much commission do an agent managing SG, TH and MY destinations obtained?

##**A.2. Objective**

The goals of this live code session are to answer questions from the problem statement using queries into company data, present the discoveries in a clear and concise manner, and to infer all the informations with analyses and conclusion.

#**B. Libraries**

Here we import the necessary function libraries for this session.

In [ ]:
!pip install mysql-connector-python

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.stats as stats
import sympy as sy
import mysql.connector

from numpy.linalg import inv, det, eig
from numpy import diag
from pylab import *
from sympy.tensor.array import derive_by_array
from google.colab import auth
from google.cloud import bigquery

def plot_vector2d(vector2d, origin=[0, 0], **options):
    return plt.arrow(origin[0], origin[1], vector2d[0], vector2d[1],
              head_width=0.2, head_length=0.3, length_includes_head=True,
              **options)

#**C. Initial Exploration**

Before we start querying data from the company, let's do a little exploration into it.

##**Codes**

In [74]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Explore_1 = '''
SELECT COUNT(*) AS Table_Length
FROM travel_insurance
'''


cur.execute(Explore_1)
result = cur.fetchall()
result
Length = pd.read_sql_query(Explore_1,Database)

In [75]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Explore_2 = '''
DESCRIBE travel_insurance
'''


cur.execute(Explore_2)
result = cur.fetchall()
result
Columns = pd.read_sql_query(Explore_2,Database)

In [76]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Explore_3 = '''
SELECT *
FROM travel_insurance
LIMIT 5
'''

cur.execute(Explore_3)
result = cur.fetchall()
result
travel_insurance_head = pd.read_sql_query(Explore_3,Database)

##**C.1. Table Length**

In [77]:
Length

,Table_Length
0,18219


##**C.2. Table Columns**

In [78]:
Columns

,Field,Type,Null,Key,Default,Extra
0,id,b'int(11)',NO,PRI,None,auto_increment
1,agency,b'varchar(255)',YES,,None,
2,agency_type,b'varchar(255)',YES,,None,
3,distribution_channel,b'varchar(255)',YES,,None,
4,product_name,b'varchar(255)',YES,,None,
5,claim,b'varchar(255)',YES,,None,
6,duration,b'int(4)',YES,,None,
7,destination,b'varchar(255)',YES,,None,
8,net_sales,b'int(5)',YES,,None,
9,commission,"b'float(5,2)'",YES,,None,


##**C.3. Table Header**

In [79]:
travel_insurance_head

,id,agency,agency_type,distribution_channel,product_name,claim,duration,destination,net_sales,commission,gender,age
0,1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29,9.57,F,81
1,2,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29,9.57,F,71
2,3,JZI,Airlines,Online,Value Plan,No,66,UNITED STATES,-121,42.35,F,44
3,4,JZI,Airlines,Online,Basic Plan,No,1,MALAYSIA,-18,6.30,M,47
4,5,KML,Travel Agency,Online,Premier Plan,No,53,NORWAY,-130,49.40,F,48


#**D. Problem Elaborations**

##**Codes**

In [91]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Query_1 = '''
SELECT DISTINCT 
agency AS Agencies
FROM travel_insurance
'''

cur.execute(Query_1)
result = cur.fetchall()
result
Agencies = pd.read_sql_query(Query_1,Database)

In [92]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Query_2 = '''
SELECT DISTINCT 
agency_type AS Agency_Types, 
COUNT(*) AS Number_of_Claims_Approved

FROM travel_insurance
WHERE claim = 'yes'
GROUP BY agency_type
ORDER BY COUNT(*) DESC
LIMIT 1
'''

cur.execute(Query_2)
result = cur.fetchall()
result
Approved_Agency_Types = pd.read_sql_query(Query_2,Database)

In [93]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Query_3 = '''
SELECT DISTINCT 
destination, 
COUNT(*) AS Number_of_Claims_Approved

FROM travel_insurance
WHERE claim = 'yes'
GROUP BY destination
ORDER BY COUNT(*) DESC
LIMIT 1
'''

cur.execute(Query_3)
result = cur.fetchall()
result
Approved_Destinations = pd.read_sql_query(Query_3,Database)

In [94]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Query_4 = '''
SELECT 
MIN(Age) AS Youngest_Insured_Age,
MAX(Age) AS Oldest_Insured_Age,
AVG(Age) AS Average_Insured_Age

FROM(
  SELECT age AS Age
  FROM travel_insurance
  WHERE claim = 'yes'
  ) AS Insured_Age
'''

cur.execute(Query_4)
result = cur.fetchall()
result
Approved_Ages = pd.read_sql_query(Query_4,Database)

In [95]:
Database = mysql.connector.connect(
    host='remotemysql.com',
    user='i9U4WY4li9',
    passwd='wzTkcKEcMw',
    database='i9U4WY4li9'
    )
cur = Database.cursor()

Query_5 = '''
SELECT SUM(commission) AS Commission_in_USD
FROM travel_insurance
WHERE (
  destination = 'SINGAPORE' OR
  destination = 'THAILAND' OR
  destination = 'MALAYSIA') AND
  claim = 'yes'
'''

cur.execute(Query_5)
result = cur.fetchall()
result
Commission_SG_TH_MY = pd.read_sql_query(Query_5,Database)


##**D.1. Which agencies filed insurance claims?**

Let's find out the agencies that filed claims to our company.

In [81]:
Agencies

,Agencies
0,CBH
1,JZI
2,KML
3,C2B
4,JWT
5,RAB
6,ART
7,CSR
8,CCR
9,ADM


There are 14 travel agencies in our partnership.

##**D.2. What agency type had the most insurance claims approved?**

These travel agencies handle different aspects of travel. What agency type received the most claim approvals? 

In [83]:
Approved_Agency_Types

,Agency_Types,Number_of_Claims_Approved
0,Airlines,584


Airlines agencies received the most approvals at 584 claims approved.

##**D.3. Which country destination received the highest number of claim approval?**

Now, which destination had the highest approval number?

In [85]:
Approved_Destinations

,destination,Number_of_Claims_Approved
0,SINGAPORE,550


It is Singapore at 550 claims approved.

##**D.4. How old are the approved claim holders?**

We want to know the ages of the approved claim holders.

In [88]:
Approved_Ages

,Youngest_Insured_Age,Oldest_Insured_Age,Average_Insured_Age
0,14,118,38.8294


The youngest person who got the claim approved is a teenager while the oldest is a supercentenarian. On average, approved claim holders age at 38 years old.

##**D.5. How much commission do an agent managing SG, TH and MY destinations obtained?**

An insurance agent is assigned to manage insurance claims for destinations to Singapore, Thailand and Malaysia. Let's see how much the agent made from commissions.

In [68]:
Commission_SG_TH_MY

,Commission_in_USD
0,16176.58


The agent made 16 thousand dollars from commissions.

#**E. Conclusion**

From the queries above, we can infer that the insurance company is holding partnership with 14 travel agencies and grants most claim approvals to airlines agencies. The claims are approved indiscriminately towards the holders' age.Most of approved claims come from trips to Singapore. An insurance agent that manages the claims to not just Singapore but also Thailand and malaysia received 16 thousand dollars in commissions from approved claims.